# Wave Equation

## Caelan Osman

## Math 437

## November 4, 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize as opt
from autograd import numpy as anp
from autograd import elementwise_grad
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Problem 1
Numerically approximate the solution to the following BVP

$$
u_{tt} = c^2 u_{xx} \\
u(0,t) = u(1,t) = 0, \\
u(x,0) = \sin(2\pi x), \\
u_t(x,0) = 0. 
$$

Numerically approximate the solution $u(x,t)$ for $t\in[0,0.5]$. Use $J=50$ subintervals in the $x$ dimension and $M=50$ subintervals in the $t$ dimension. Animate the results. Compare you results with the analytic solution $u(x,t) = \sin(2\pi x)\cos(2\pi t).$ This function is know as a standing wave.

In [2]:
def wave_ivp(x0, xf, t0, tf, c_2, u_bv, f, g, J=50, M=50):
    ''' this solves the initial value wave equation. Note this is for a periodice equation
        that is u(L, t) = u(R, t) for all t where R and L are the right and Left endpoints
        of the boundary respectively. 
        Parameters: 
            x0, xf (float): the spatial domain
            t0, tf (float): the time domain
            c_2 (float): the squared constant in the advection equation
            u_bv (callable): the function that describes the boundary conditions
            f (callable): the function at u(x, 0)
            g (callable): the derivative u_t(x, 0)
            J, M (float): how may discrete points to use in x and t respectively
        Returns:
            x_vals ((J, ) np.ndarray): the discrete x_values on the spatial domain
            U ((M, J) np.ndarray): where each row contains the output u at (x_j, t_m)
    '''
    s = np.abs(c_2)
    x_vals = np.linspace(x0, xf, J+1)
    t_vals = np.linspace(t0, tf, M+1)
    #need to specify CFL condition now
    deltaX = x_vals[1] - x_vals[0]
    deltaT = t_vals[1] - t_vals[0]
    #ge CFL condition
    CFL = s * deltaT / deltaX

    #define A matrix
    A_diag = np.diag(2*(1 - CFL**2)*np.ones(J-1), k=0)
    A_sup_diag = np.diag(CFL**2 *np.ones(J-2), k=1)
    A_sub_diag = np.diag(CFL**2 *np.ones(J-2), k=-1)
    A = A_diag + A_sup_diag + A_sub_diag

    #define numerical second diff of f
    f_pp = elementwise_grad(elementwise_grad(f))
    #the first two values
    U_j_0 = f(x_vals[1:J])
    U_j_1 = U_j_0 + g(x_vals[1:J])*deltaT + c_2 * f_pp(x_vals[1:J]) * deltaT**2/2 


    #now we can find our U matrix
    U = np.vstack((U_j_0, U_j_1))
    for i in range(2, M+1):
        U_m = A @ U[i-1] - U[i-2]
        U = np.vstack((U, U_m))

    #calculate the left and right boundary condtion
    left_bc = u_bv(x_vals[0], t_vals).reshape(-1, 1)
    right_bc = u_bv(x_vals[-1], t_vals).reshape(-1, 1)
    U = np.hstack((left_bc, U, right_bc))


    return x_vals, U

In [3]:
def prob1_vals():
    x0, xf, t0, tf, c_2 = 0, 1, 0, 1/2., 1
    u_bv = lambda x, t: 0.0*t
    f = lambda x: anp.sin(2*anp.pi*x)
    g = lambda x: 0.0*x
    x_vals, U = wave_ivp(x0, xf, t0, tf, c_2, u_bv, f, g, J=50, M=50)

    return x_vals, U

x_vals, U = prob1_vals()
#now we create our animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(100)
ax = fig.add_subplot(111)
ax.set_xlim(0, 1)
plt.suptitle(r'Standing Wave')
#update function for animation
def update(i):
    ax.clear()
    ax.set_ylim(-1, 1)
    ax.plot(x_vals, U[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax
#create animation 
ani = animation.FuncAnimation(fig, update, frames = U.shape[0], interval=25)
HTML(ani.to_html5_video())

# Problem 2

Consider the boundary value problem 

$$
u_{tt} = u_{xx}, \\
u(0,t) = u(1,t) = 0, \\
u(x,0) = 0.2e^{-m^2(x-1/2)^2} \\
u_t(x,0) = 0.4m^2(x-\frac{1}{2})e^{-m^2(x-1/2)^2}.
$$

The solution of this problem is ta Gaussian pulse. It travels to the right at a constant speed. This solution models, for example, a wave pulse in a stretched string. Note that the fixed boundary conditions reflect the pulse back when it meets the boundary.

Numerically approximate the solution $u(x,t)$ for $t\in[0,1]$. Set $m=20$. Use 200 subintervals in space and 220 in time, and animate your results. Then use 200 subintervals in space and 180 in time, and animate your results. Note that the stability condition is not satisfied for the second mesh.

In [4]:
def prob2_vals():
    x0, xf, t0, tf, c_2 = 0, 1, 0, 1, 1
    u_bv = lambda x, t: 0.0*t
    f = lambda x: 1/5 * anp.exp(-(20*(x - 1/2.))**2)
    g = lambda x: 2/5 * 20**2* (x - 1/2)* anp.exp(-(20*(x - 1/2.))**2)
    x_vals_1, U_1 = wave_ivp(x0, xf, t0, tf, c_2, u_bv, f, g, J=200, M=220)
    x_vals_2, U_2 = wave_ivp(x0, xf, t0, tf, c_2, u_bv, f, g, J=200, M=180)


    return x_vals_1, U_1, x_vals_2, U_2

x_vals_1, U_1, x_vals_2, U_2 = prob2_vals()
#now we create our animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(100)
ax = fig.add_subplot(111)
ax.set_xlim(0, 1)
plt.suptitle(r'Stable Condition')
#update function for animation
def update(i):
    ax.clear()
    ax.set_ylim(-1, 1)
    ax.set_yticks([])
    ax.plot(x_vals_1, U_1[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax
#create animation 
ani = animation.FuncAnimation(fig, update, frames = U_1.shape[0], interval=25)
HTML(ani.to_html5_video())

In [5]:
#now we create our animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(100)
ax = fig.add_subplot(111)
ax.set_xlim(0, 1)
plt.suptitle(r'Unstable Condition')
#update function for animation
def update(i):
    ax.clear()
    ax.set_ylim(-1, 1)
    ax.set_yticks([])
    ax.plot(x_vals_2, U_2[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax
#create animation 
ani = animation.FuncAnimation(fig, update, frames = U_2.shape[0], interval=25)
HTML(ani.to_html5_video())

# Problem 3

Consider the initial boundary value problem
$$
	u_{tt} = u_{xx}, \\
	u(0,t) = u(1,t) = 0, \\
	u(x,0) = 0.2e^{-m^2(x-1/2)^2}\\
	u_t(x,0) = 0.
$$
The initial condition separates into two smaller, slower-moving pulses, one travelling to the right and the other to the left.
This solution models, for example, a plucked guitar string

Numerically approximate the solution $u(x,t)$ for $t \in \left[0,2\right]$.
Set $m=20$.
Use 200 subintervals in space and 440 in time, and animate your results.
It is rather easy to see that the solution to this problem is the sum of two travelling waves, one travelling to the left and the other to the right, as described earlier.

In [6]:
def prob3_vals():
    x0, xf, t0, tf, c_2 = 0, 1, 0, 2, 1
    u_bv = lambda x, t: 0.0*t
    f = lambda x: 1/5 * anp.exp(-(20*(x - 1/2.))**2)
    g = lambda x: 0.0*x
    x_vals_3, U_3 = wave_ivp(x0, xf, t0, tf, c_2, u_bv, f, g, J=200, M=440)


    return x_vals_3, U_3

x_vals_3, U_3 = prob3_vals()

#now we create our animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(100)
ax = fig.add_subplot(111)
ax.set_xlim(0, 1)
plt.suptitle(r'Traveling Waves')
#update function for animation
def update(i):
    ax.clear()
    ax.set_ylim(-1, 1)
    ax.plot(x_vals_3, U_3[i])
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    return ax
#create animation 
ani = animation.FuncAnimation(fig, update, frames = U_3.shape[0], interval=25)
HTML(ani.to_html5_video())

# Problem 4

Consider the initial boundary value problem

$$
	u_{tt} = u_{xx}, \\
	u(0,t) = u(1,t) = 0, \\
	u(x,0) = \begin{cases} 1/3 & \text{if } 5/11 < x < 6/11,\\
	0 & \text{otherwise}
	\end{cases}\\
	u_t(x,0) = 0.
$$

Numerically approximate the solution $u(x,t)$ for $t \in \left[0, 2\right]$.
Use 200 subintervals in space and 440 in time, and animate your results.
Even though the method is second order and stable for this discretization, since the initial condition is discontinuous there are large dispersive errors.

In [7]:
def prob4_vals():
    x0, xf, t0, tf, c_2 = 0, 1, 0, 2, 1
    u_bv = lambda x, t: 0.0*t
    def f(x):
        x_vals = np.atleast_1d(x)
        return np.array([1/3 if 5/11 < x and x < 6/11 else 0 for x in x_vals])
        
    g = lambda x: 0.0*x
    x_vals_4, U_4 = wave_ivp(x0, xf, t0, tf, c_2, u_bv, f, g, J=200, M=440)


    return x_vals_4, U_4

x_vals_4, U_4 = prob4_vals()

#now we create our animation
plt.ioff()
fig = plt.figure()
fig.set_dpi(100)
ax = fig.add_subplot(111)
ax.set_xlim(0, 1)
plt.suptitle(r'Unstable Traveling Waves')
#update function for animation
def update(i):
    ax.clear()
    ax.set_ylim(-1, 1)
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    ax.plot(x_vals_4, U_4[i])
    return ax
#create animation 
ani = animation.FuncAnimation(fig, update, frames = U_4.shape[0], interval=25)
HTML(ani.to_html5_video())

/home/caelan/anaconda3/lib/python3.8/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


# Problem 5


Numerically solve the initial value problem

$$
	u_t -su_x + uu_x = u_{xx}, \quad x \in (-\infty,\infty),\\
	u(x,0) = v(x),
$$

for $t \in [0,1]$.
Let the perturbation $v(x)$ be given by

$$
v(x) = 3.5(\sin{(3x)} + 1)\frac{1}{\sqrt{2\pi}} \exp{(-x^2/2)}
$$

And let the initial condition be $u(x, 0) = \hat{u}(x) + v(x)$
Approximate the $x$ domain,$(-\infty, \infty)$, numerically by the finite interval $[-20,20]$, and fix $u(-20) = u_-$, $u(20) = u_+$. Let $u_- = 5$, $u_+ = 1$.
Use 150 intervals in space and 350 steps in time.
Animate your results.
You should see the solution converge to a translate of the travelling wave $\hat{u}$.

Hint: This difference scheme is no longer a linear equation.
We have a nonlinear equation in $U^{n+1}$.
We can still solve this function using Newton's method or some other similar solver.
In this case, use `scipy.optimize.fsolve`.